In [17]:
import numpy as np
import doubleml as dml
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from doubleml.datasets import make_pliv_CHS2015
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.base import clone
from doubleml import DoubleMLData
from doubleml.double_ml_samplesel import DoubleMLSS
from doubleml.double_ml_selection import DoubleMLS

from sklearn.linear_model import LassoCV

from doubleml.double_ml import DoubleML
from doubleml.double_ml_data import DoubleMLData

## Load data

In [18]:
data = pd.read_csv("./data_JC.csv")
data.head()

,treat,emply4,jcmsa_1,jcmsa_2,age,race_eth_1,race_eth_2,race_eth_3,ntv_lang_1,ntv_lang_2,...,st_hwh97,st_hwh98,st_hwh99,st_hwh100,st_hwh101,st_hwh102,st_hwh103,st_hwh104,hourwag_52,hourwag_y4
0,13,-1,0,1,0.357121,0,0,1,0,0,...,-0.425291,-0.426362,-0.428669,-0.429545,-0.433552,-0.432121,-0.436146,-0.438621,0.0,0.000000
1,0,1,0,0,1.319299,0,1,0,0,0,...,0.507288,0.502377,0.504842,0.498681,0.497172,-0.432121,0.182607,0.181539,0.0,7.007009
2,31,1,0,1,-0.169031,0,0,0,0,0,...,-0.425291,-0.426362,-0.428669,-0.429545,-0.433552,-0.432121,-0.436146,-0.438621,6.5,7.618556
3,0,1,0,0,-0.622194,0,0,0,0,0,...,0.351858,0.347587,-0.428669,-0.429545,-0.433552,-0.432121,-0.436146,-0.438621,5.7,6.787174
4,0,1,0,0,-0.128411,0,1,0,0,0,...,-0.425291,-0.426362,-0.428669,-0.429545,-0.433552,-0.432121,-0.436146,-0.438621,0.0,5.137620


#######################################
### EVALUATION OF ACADEMIC TRAINING ###
#######################################

In [19]:
# Selection into treatment/control status for the female group
subsampleF = data[data['female'] == 1]
subsample = subsampleF[subsampleF['treat'].isin([11, 12, 13, 21, 22, 23])]

# Selection into treatment
subsample['d'] = None
subsample.loc[subsample['treat'].isin([21, 22, 23]), 'd'] = 1
subsample.loc[subsample['treat'].isin([11, 12, 13]), 'd'] = 0
print(subsample['d'].isna().sum())

# Splitting by treatment
T1 = subsample[subsample['d'] == 1]
T0 = subsample[subsample['d'] == 0]

0


/tmp/ipykernel_293930/1468549734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample['d'] = None


In [20]:
print("Mean (workh52_0) for treatment group: ", T1['workh52_0'].mean(), '\n',
      "Standard Deviation (workh52_0) for treatment group: ", T1['workh52_0'].std())
print("Mean (workh52_0) for control group: ", T0['workh52_0'].mean(), '\n',
      "Standard Deviation (workh52_0) for control group: ", T0['workh52_0'].std())

print("Mean (hourwag_52): ", subsample['hourwag_52'].mean(), '\n',
      "Standard Deviation (hourwag_52): ", subsample['hourwag_52'].std())
print("Mean (workh52_0): ", subsample['workh52_0'].mean(), '\n',
      "Standard Deviation (workh52_0): ", subsample['workh52_0'].std())

Mean (workh52_0) for treatment group:  0.3072289156626506 
 Standard Deviation (workh52_0) for treatment group:  0.4616232785224906
Mean (workh52_0) for control group:  0.335 
 Standard Deviation (workh52_0) for control group:  0.4731748855991927
Mean (hourwag_52):  1.6686438544660196 
 Standard Deviation (hourwag_52):  2.9278099018848307
Mean (workh52_0):  0.312621359223301 
 Standard Deviation (workh52_0):  0.46378667301484416


In [21]:
# Selection into employment by treatment status
subsample['s'] = subsample['workh52_0']
subsample['y'] = subsample['hourwag_52']

# Selection into employment by treatment - Linear Regression
relation = smf.ols('s ~ d', data=subsample).fit()
print(relation.summary())

                            OLS Regression Results                            
Dep. Variable:                      s   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5776
Date:                Fri, 16 Feb 2024   Prob (F-statistic):              0.447
Time:                        12:05:49   Log-Likelihood:                -669.34
No. Observations:                1030   AIC:                             1343.
Df Residuals:                    1028   BIC:                             1353.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3350      0.033     10.213      0.0

/tmp/ipykernel_293930/3858797875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample['s'] = subsample['workh52_0']
/tmp/ipykernel_293930/3858797875.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample['y'] = subsample['hourwag_52']


In [22]:
columns = ["jcmsa_1", "jcmsa_2", "age", "race_eth_1", "race_eth_2", "race_eth_3","ntv_lang_1", "ntv_lang_2", "hh14_1", "hh14_2", 
                       "hh14_3", "hh14_4", "welf_kid_1", "welf_kid_2", "welf_kid_3", "m_work14_0", 
                       "occ_moth_1","occ_moth_2", "occ_moth_3", "occ_moth_4", "occ_moth_5", "occ_moth_6", "occ_fath_1", "occ_fath_2", "occ_fath_3", "occ_fath_5", 
                       "occ_fath_6", "occ_fath_7", "marriage_1", "marriage_2", "marriage_3", "haschld_0", "proplive_1", "proplive_2", "pregn_ra_1", "pregn_ra_2", "old", 
                       "yng", "othwith_1", "othwith_2", "othwith_3", "othwith_4", "ageparnt", "numb_hh", "r_head_0", "hhmemb_1", "hhmemb_2", "hhmemb_3", 
                       "hhmemb_4", "hhmemb_5", "hous_arr_1", "hous_arr_2", "hous_arr_3", "pay_rent_0", "hgc", "hgc_moth", "hgc_fath", "hs_d_0", "ged_d_0", "voc_d_0", 
                       "oth_deg_0", "inschool_0", "any_ed1_0", "n_ed_cat_1", "n_ed_cat_2", "monined", "reasleft_1", "reasleft_2", "reasleft_3", "reasleft_4", "reasleft_5", 
                       "reasleft_6", "reasleft_7", "reasleft_8", "rec_ed_1",  "rec_ed_2", "rec_ed_3", "rec_ed_4", "rec_ed_5", "rec_ed_6", "rec_ed_7", "typeed_r_1", "typeed_r_2", 
                       "typeed_r_3", "typeed_r_4", "typeed_r_5", "typeed_r_6", "nhrsed_r", "reased_r_1", "numbjobs", "evworkb_0", "yr_work1_0", 
                       "earn_yr", "mosinjob", "rec_job_1","rec_job_2", "rec_job_3", "rec_job_4", "occ_r_1", "occ_r_2", "occ_r_3", "occ_r_4", "occ_r_5", "occ_r_6", "occ_r_7", "occ_r_8", 
                       "hrswk_jr", "hrwager", "coop_r_0", "govprg_r_0", "leftjobr_0", "rslftjr_1", "rslftjr_2", "rslftjr_3", "rslftjr_4", "rslftjr_5", "rslftjr_6", "rslftjr_7", 
                       "rslftjr_8", "mos_afdc", "mos_othw", "mos_fs", "got_anyw_0", "mos_anyw", "gotafdc1_0", "gotothw1_0", "gotfs1_0", "hh_inc_1", "hh_inc_2", "hh_inc_3", 
                       "hh_inc_4", "pers_inc_1", "pers_inc_2", "pers_inc_3", "health_1", "health_2", "health_3", "sick_0", "typehlth_1", "typehlth_2", "typehlth_3", "typehlth_4", 
                       "typehlth_5", "py_cig_0", "ev_alchl_0", "ev_pot_0", "py_pot_0", "ev_coke_0", "py_coke_0", "ev_crack_0", "py_crack_0", "ev_hroin_0", "py_hroin_0", "ev_speed_0", 
                       "py_speed_0", "py_lsd_0", "ev_lsd_0", "ev_othdr_0", "py_othdr_0", "ev_injct_0", "drug_trt_0",  "mout_trt", "mos_trtr_1", "mos_trtr_2", "mos_trtr_3", 
                       "narrcat_1", "narrcat_2", "narrcat_3", "evarrst1_0", "rc_arrst_1", "rc_arrst_2", "rc_arrst_3", "rc_arrst_4",
                       "marrcat1_1", "marrcat1_2", "agearcat_1", "agearcat_2", "agearcat_3", "burglary_0", "robbery_0", 
                       "assault_0", "larceny_0", "drugviol_0", "othpers_0", "othmisc_0", "sercr_s1_0", "sercr_s3_0", "sercr_s4_0", "sercr_s5_0", "sercr_s6_0", "sercr_s7_0", "n_guilty", 
                       "guilty2_0", "wksjail", "pending2_0", "copplea2_0", "sercr_c1_0", "sercr_c2_0", "sercr_c3_0", "sercr_c5_0", "sercr_c6_0", "sercr_c7_0", "asslt_c2_0", "rob_c2_0", 
                       "burgl_c2_0", "drviolc2_0", "othperc2_0", "othmscc2_0", "evjail2_0", "parole2_0", "hear_jc_1", "hear_jc_2", "hear_jc_3", "hear_jc_4", "hear_jc_5", 
                       "hear_jc_6", "hear_jc_7", "from_oa_0", "knew_jc_0", "info_jc_1", "info_jc_2", "info_jc_3", "info_jc_4", "info_jc_5", "info_jc_6", "info_jc_7", "r_home_0", "r_comm_0", "r_train_0", 
                       "r_crgoal_0", "r_getged_0", "r_nowork_0", "r_other_0", "mostimpr_1", "mostimpr_2", "mostimpr_3", "mostimpr_4", "mostimpr_5", "mostimpr_6", "mostimpr_7", "othimpr_1", 
                       "othimpr_2", "othimpr_3", "othimpr_4", "othimpr_5", "othimpr_6", "e_math_0", "e_read_0", "e_along_0", "e_contrl_0", "e_esteem_0", "e_spcjob_0", "e_friend_0", "knewcntr_0", 
                       "imprcntr_1", "imprcntr_2", "imprcntr_3", "imprcntr_4", "imprcntr_5", "imprcntr_6", "imprcntr_7", "imprcntr_8", "knewjob_0", "typejobb_1", "typejobb_2", "typejobb_3", 
                       "typejobb_4", "typejobb_5", "typejobb_6", "typejobb_7", "typejobb_8", "typejobb_9", "typejobb_10", "earn_cmp", "hadworry_0", "typeworr_1", "typeworr_2", "typeworr_3", 
                       "typeworr_4", "typeworr_5", "typeworr_6", "typeworr_7", "typeworr_8", "typeworr_9", "talk_par_0", "imp_par_0", "encr_par_0", "talk_rel_0", "imp_rel_0", "encr_rel_0", 
                       "talk_frd_0", "imp_frd_0", "encr_frd_0", "talk_tch_0", "imp_tch_0", "encr_tch_0", "talk_cw_0", "imp_cw_0", "encr_cw_0", "talk_pro_0", "imp_pro_0", "encr_pro_0", 
                       "talk_chl_0", "imp_chl_0", "encr_chl_0", "talk_adl_0", "imp_adl_0", "encr_adl_0", "encr_jcr_1", "encr_jcr_2", "encr_jcr_3", "howspoke_0", "telemode_1","telemode_2", 
                       "placeipc_1", "placeipc_2", "placeipc_3", "placeipc_4", "placeipc_5", "placeipc_6", "talkstay_0", "way_stay_0", "talkvstf_0", "talktold_0", "tradwant_0", "chncetrd_1", 
                       "chncetrd_2", "chncetrd_3", "totalhrs", "vstf_cat_1", "vstf_cat_2", "schl52_0", "trng1_0", "trng2_0", "trng52_0", "welf1_0", "welf3_0", "welf4_0", "welf5_0", "welf6_0", "welf7_0", 
                       "welf8_0", "welf9_0", "welf10_0", "welf11_0", "welf12_0", "currjob_0", "nchld"]

In [24]:
dml_data = DoubleMLData(subsample,
                        y_col = 'y',
                        d_cols = 'd',
                        t_col='s',
                        x_cols = columns)

# Define ML models for nuisance functions
learner = RandomForestRegressor(n_estimators = 50, max_features = 'sqrt', max_depth= 5)
learner_class = RandomForestRegressor(n_estimators = 50, max_features = 'sqrt', max_depth= 5)
ml_mu_sim = clone(learner)
ml_pi_sim = clone(learner_class)
ml_p_sim = clone(learner_class)

In [25]:
academic_training = DoubleMLS(dml_data, ml_mu_sim, ml_pi_sim, ml_p_sim)
academic_training.fit()

print(academic_training)

================== DoubleMLS Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['jcmsa_1', 'jcmsa_2', 'age', 'race_eth_1', 'race_eth_2', 'race_eth_3', 'ntv_lang_1', 'ntv_lang_2', 'hh14_1', 'hh14_2', 'hh14_3', 'hh14_4', 'welf_kid_1', 'welf_kid_2', 'welf_kid_3', 'm_work14_0', 'occ_moth_1', 'occ_moth_2', 'occ_moth_3', 'occ_moth_4', 'occ_moth_5', 'occ_moth_6', 'occ_fath_1', 'occ_fath_2', 'occ_fath_3', 'occ_fath_5', 'occ_fath_6', 'occ_fath_7', 'marriage_1', 'marriage_2', 'marriage_3', 'haschld_0', 'proplive_1', 'proplive_2', 'pregn_ra_1', 'pregn_ra_2', 'old', 'yng', 'othwith_1', 'othwith_2', 'othwith_3', 'othwith_4', 'ageparnt', 'numb_hh', 'r_head_0', 'hhmemb_1', 'hhmemb_2', 'hhmemb_3', 'hhmemb_4', 'hhmemb_5', 'hous_arr_1', 'hous_arr_2', 'hous_arr_3', 'pay_rent_0', 'hgc', 'hgc_moth', 'hgc_fath', 'hs_d_0', 'ged_d_0', 'voc_d_0', 'oth_deg_0', 'inschool_0', 'any_ed1_0', 'n_ed_cat_1', 'n_ed_cat_2', 'mon

#######################################
### EVALUATION OF VOCATIONAL TRAINING ###
#######################################

In [26]:
subsample = subsampleF[subsampleF['treat'].isin([11, 12, 13, 31, 32, 33])]

# Selection into treatment
subsample['d'] = np.nan
subsample.loc[subsample['treat'].isin([31, 32, 33]), 'd'] = 1
subsample.loc[subsample['treat'].isin([11, 12, 13]), 'd'] = 0
print(subsample['d'].isna().sum())

# Splitting by treatment
T1 = subsample[subsample['d'] == 1]
T0 = subsample[subsample['d'] == 0]

0


/tmp/ipykernel_293930/3903341877.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample['d'] = np.nan


In [11]:
print("Mean (workh52_0) for treatment group: ", T1['workh52_0'].mean(), '\n',
      "Standard Deviation (workh52_0) for treatment group: ", T1['workh52_0'].std())
print("Mean (workh52_0) for control group: ", T0['workh52_0'].mean(), '\n',
      "Standard Deviation (workh52_0) for control group: ", T0['workh52_0'].std())

print("Mean (hourwag_52): ", subsample['hourwag_52'].mean(), '\n',
      "Standard Deviation (hourwag_52): ", subsample['hourwag_52'].std())
print("Mean (workh52_0): ", subsample['workh52_0'].mean(), '\n',
      "Standard Deviation (workh52_0): ", subsample['workh52_0'].std())

Mean (workh52_0) for treatment group:  0.36773428232502964 
 Standard Deviation (workh52_0) for treatment group:  0.4824747817849973
Mean (workh52_0) for control group:  0.335 
 Standard Deviation (workh52_0) for control group:  0.4731748855991927
Mean (hourwag_52):  2.0218169680728666 
 Standard Deviation (hourwag_52):  3.0606928974866285
Mean (workh52_0):  0.36145733461169705 
 Standard Deviation (workh52_0):  0.4806531313524103


In [27]:
# Selection into employment by treatment status
subsample['s'] = subsample['workh52_0']
subsample['y'] = subsample['hourwag_52']

# Selection into employment by treatment - Linear Regression
relation = smf.ols('s ~ d', data=subsample).fit()
print(relation.summary())

                            OLS Regression Results                            
Dep. Variable:                      s   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.7496
Date:                Fri, 16 Feb 2024   Prob (F-statistic):              0.387
Time:                        12:07:22   Log-Likelihood:                -714.97
No. Observations:                1043   AIC:                             1434.
Df Residuals:                    1041   BIC:                             1444.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3350      0.034      9.855      0.0

/tmp/ipykernel_293930/3858797875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample['s'] = subsample['workh52_0']
/tmp/ipykernel_293930/3858797875.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample['y'] = subsample['hourwag_52']


In [28]:
# X columns are the same as before
dml_data = DoubleMLData(subsample,
                        y_col = 'y',
                        d_cols = 'd',
                        t_col='s',
                        x_cols = columns)

# Define ML models for nuisance functions
learner = RandomForestRegressor(n_estimators = 50, max_features = 'sqrt', max_depth= 5)
learner_class = RandomForestRegressor(n_estimators = 50, max_features = 'sqrt', max_depth= 5)
ml_mu_sim = clone(learner)
ml_pi_sim = clone(learner_class)
ml_p_sim = clone(learner_class)

In [29]:
vocational_training = DoubleMLS(dml_data, ml_mu_sim, ml_pi_sim, ml_p_sim)
vocational_training.fit()

print(vocational_training)

================== DoubleMLS Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['jcmsa_1', 'jcmsa_2', 'age', 'race_eth_1', 'race_eth_2', 'race_eth_3', 'ntv_lang_1', 'ntv_lang_2', 'hh14_1', 'hh14_2', 'hh14_3', 'hh14_4', 'welf_kid_1', 'welf_kid_2', 'welf_kid_3', 'm_work14_0', 'occ_moth_1', 'occ_moth_2', 'occ_moth_3', 'occ_moth_4', 'occ_moth_5', 'occ_moth_6', 'occ_fath_1', 'occ_fath_2', 'occ_fath_3', 'occ_fath_5', 'occ_fath_6', 'occ_fath_7', 'marriage_1', 'marriage_2', 'marriage_3', 'haschld_0', 'proplive_1', 'proplive_2', 'pregn_ra_1', 'pregn_ra_2', 'old', 'yng', 'othwith_1', 'othwith_2', 'othwith_3', 'othwith_4', 'ageparnt', 'numb_hh', 'r_head_0', 'hhmemb_1', 'hhmemb_2', 'hhmemb_3', 'hhmemb_4', 'hhmemb_5', 'hous_arr_1', 'hous_arr_2', 'hous_arr_3', 'pay_rent_0', 'hgc', 'hgc_moth', 'hgc_fath', 'hs_d_0', 'ged_d_0', 'voc_d_0', 'oth_deg_0', 'inschool_0', 'any_ed1_0', 'n_ed_cat_1', 'n_ed_cat_2', 'mon